# Users and Mentions grouped bar charts  - Fake Covid-19 dataset

We've used the following packages:

In [1]:
import pandas as pd
import numpy as np
import json
import sys
import string
import re
import itertools  
from emot.emo_unicode import UNICODE_EMO, EMOTICONS
import emoji
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt 
import nltk
from PIL import Image
import altair as alt
import csv
import itertools

In order to do the classification of the tweets, we need to read the csv file and the json file:

In [2]:
csv_dataframe = pd.read_csv('dataset/FINAL_fakecovid_final_filtered_dataset_clean.csv',sep=";")
csv_dataframe['tweet_id'] = csv_dataframe['tweet_id'].astype(str)
csv_list = csv_dataframe.values.tolist()
lista_unica_csv=list(itertools.chain.from_iterable(csv_list))

data = []
with open('dataset/fakecovid_result_final_translated_full.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))

In this notebook we are going to visualize the users that have created the tweets and the usersname mentioned in the tweets. Every bar chart here is stacked because we're going to classify every tweet with a category (False or Partially False), according to the dataset.

In [3]:
index=0
cmt_list = []
#stopwords = set(STOPWORDS) 
for element in data:
    token_id = data[index]['id_str']
    indice_csv = lista_unica_csv.index(token_id)
    token = data[index]['user']['screen_name']
    final_token = token + " " +lista_unica_csv[indice_csv+1].lower().replace(" ", "")
    cmt_list.append(final_token)
    index=index+1

In [4]:
index_mentions=0
cmt_list_mentions = []
for element in data:
    token_id_mentions = data[index_mentions]['id_str']
    indice_csv_mentions = lista_unica_csv.index(token_id_mentions)
    for entity in data[index_mentions]['entities']['user_mentions']:
        token_mentions=entity['screen_name']
        final_token_mentions = token_mentions + " " +lista_unica_csv[indice_csv_mentions+1].lower().replace(" ", "")
        cmt_list_mentions.append(final_token_mentions)
    index_mentions=index_mentions+1

## Users

We create the pandas DataFrame and then we work on it in order to create a better bar chart:

In [5]:
fdist = dict(nltk.FreqDist(cmt_list))
df = pd.DataFrame.from_dict(fdist, orient='index').reset_index()
df = df.rename(columns={'index':'usernames', 0:'count'})
col_one_list = df['usernames'].tolist()
col_two_list = df['count'].tolist()

typelist=[]
namelist=[]

index = 0

count_false = [0] * len(col_one_list)
count_part = [0] * len(col_one_list)

for el in col_one_list:
    tok = el.split()
    namelist.append(tok[0])
    #typelist.append(tok[1])
    if tok[0] in namelist:
        indx = namelist.index(tok[0])
        if tok[1] == "false":
            count_false[indx] = col_two_list[index]
        elif tok[1] == "partiallyfalse":
            count_part[indx] = col_two_list[index]
        else:
            print("errore count")
   
    index = index + 1


df['Usernames']=namelist
df['False']=count_false
df['Partially False']=count_part
del df['count'] #clean column
df['count'] = df['False'] + df['Partially False']

#print(df.to_string())
#df['type']=typelist

#df = df.sort_values(by=['count'],ascending=[False])

We visualize users with the most number of tweets created, we have filtered in the range of a minum of 5 tweets to a maxium to 35 tweets total (which is the maxium number of tweets created by a user in the dataset). Every bar is colored basing on the amount of tweet written per each category (False, Partually False).

In [6]:
bars = alt.Chart(df).transform_fold(
    ['False', 'Partially False'],
).mark_bar().encode(
    x=alt.X('key:N',title=None),
    y=alt.Y('value:Q',title='Tweet count'),
    color=alt.Color('key:N',scale=alt.Scale(range=['#903C7C','#BDBD3F']),title="Category"),
    column=alt.Column('Usernames:N'),
    tooltip=[
        alt.Tooltip('key:N', title='Tweet Type'),
        alt.Tooltip('Usernames:N', title='Username'),
        alt.Tooltip('value:Q', title='Tweet count')
    ]
).transform_filter(
    alt.FieldRangePredicate(field='count', range=[7, 35])
).properties(
    title="The users with the most number of tweets created classified by category (False, Partually False)",
    width=95
).configure_title(
    fontSize=17,
    offset=25
).configure_axis(
    labelFontSize=13,
    titleFontSize=15,
    titlePadding=15
).configure_legend(
    titleFontSize=15,
    labelFontSize=13,
    titlePadding=10
).configure_header(
    labelFontSize=14,
    titleFontSize=16
)

bars

alt.Chart(...)

## Mentions

We create the pandas DataFrame and then we work on it in order to create a better bar chart:

In [7]:
fdist_mentions = dict(nltk.FreqDist(cmt_list_mentions))
#print(fdist_mentions)
df_mentions = pd.DataFrame.from_dict(fdist_mentions, orient='index').reset_index()
df_mentions = df_mentions.rename(columns={'index':'usernames', 0:'count'})
col_one_list_mentions = df_mentions['usernames'].tolist()
col_two_list_mentions = df_mentions['count'].tolist()

#print(df_mentions)

typelist_mentions=[]
namelist_mentions=[]

indexm = 0

count_false_mentions = [0] * len(col_one_list_mentions)
count_part_mentions = [0] * len(col_one_list_mentions)

for el in col_one_list_mentions:
    tokm = el.split()
    namelist_mentions.append(tokm[0])
    #typelist.append(tok[1])
    if tokm[0] in namelist_mentions:
        indxm = namelist_mentions.index(tokm[0])
        if tokm[1] == "false":
            count_false_mentions[indxm] = col_two_list_mentions[indexm]
        elif tokm[1] == "partiallyfalse":
            count_part_mentions[indxm] = col_two_list_mentions[indexm]
        else:
            print("errore count")
   
    indexm = indexm + 1


#print("count false len "+str(len(count_false)))
df_mentions['Mentioned Usernames']=namelist_mentions
df_mentions['False']=count_false_mentions
df_mentions['Partially False']=count_part_mentions
del df_mentions['count']
del df_mentions['usernames']
df_mentions['count'] = df_mentions['False'] + df_mentions['Partially False']

#df['type']=typelist

#print(df_mentions.to_string())

We visualize the most mentioned users classified by the category of the tweet that they're mentioned in, we have filtered in the range of a minum of 4 tweets to a maxium to 22 tweets total (which is the maxium number of mentions per users in the dataset). Every bar is colored basing on the amount of tweet written per each category (False, Partually False).

In [8]:
barsm = alt.Chart(df_mentions).transform_fold(
    ['False', 'Partially False'],
).mark_bar().encode(
    x=alt.X('key:N',title=None),
    y=alt.Y('value:Q',title="Tweet count"),
    color=alt.Color('key:N',scale=alt.Scale(range=['#006CD1','#D47411']),title="Category"),
    column=alt.Column('Mentioned Usernames:N'),
    tooltip=[
        alt.Tooltip('key:N', title='Tweet Type'),
        alt.Tooltip('Mentioned Usernames:N', title='Mentioned Username'),
        alt.Tooltip('value:Q', title='Tweet count')
    ]
).transform_filter(
    alt.FieldRangePredicate(field='count', range=[6, 22])
).properties(
    title="The most mentioned users classified by the category of the tweet that they're mentioned in",
    width=95
).configure_title(
    fontSize=17,
    offset=25
).configure_axis(
    labelFontSize=13,
    titleFontSize=15,
    titlePadding=15
).configure_legend(
    titleFontSize=15,
    labelFontSize=13,
    titlePadding=10
).configure_header(
    titleFontSize=16,
    labelFontSize=14
)

barsm

alt.Chart(...)